<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    <b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b>
</h1>

---

<h1 style="text-align: center;">
    Домашнее задание: градиентный спуск и линейные модели
</h1>

In [ ]:
import numpy as np
import pandas as pd
import scipy
from matplotlib import pylab, gridspec, pyplot as plt

%matplotlib inline

plt.style.use('fivethirtyeight')

## Градиентный спуск: повторение

Рассмотрим функцию от двух переменных:

In [ ]:
def f(x1, x2):
    return np.sin(x1) ** 2 + np.sin(x2) ** 2

**Напоминание.**

Что мы хотим? Мы хотим найти минимум этой функции (в машинном обучении мы обычно хотим найти минимум **функции потерь**, например, **MSE**), а точнее найти $x_1$ и $x_2$ такие, что при них значение $f(x_1,x_2)$ минимально, то есть **точку экстремума**. 

Как мы будем искать эту точку? Используем методы оптимизации (в нашем случае - минимизации). Одним из таких методов и является **градиентный спуск**.

Реализуем функцию, которая будет осуществлять градиентный спуск для функции $f$.

**Примечание:** вам нужно посчитать частные производные именно **аналитически** и **переписать их в код**, а не считать производные численно (через отношение приращения функции к приращению аргумента) - в этих двух случаях могут различаться ответы, поэтому будьте внимательны.

In [ ]:
def grad_descent(lr, num_iter=100):
    """
    Функция, которая реализует градиентный спуск в минимум для функции f от двух переменных. 
        param lr - learning rate алгоритма
        param num_iter - количество итераций градиентного спуска
    """
    
    global f
    # в начале градиентного спуска инициализируем значения x1 и x2 какими-нибудь числами
    cur_x1, cur_x2 = 1.5, -1
    # будем сохранять значения аргументов и значений функции в процессе градиентного спуска в переменную steps
    steps = []
    
    # итерация цикла - шаг градиентнго спуска
    for iter_num in range(num_iter):
        steps.append([cur_x1, cur_x2, f(cur_x1, cur_x2)])
        
        # чтобы обновить значения cur_x1 и cur_x2, как мы помним с последнего занятия, 
        # нужно найти производные (градиенты) функции f по этим переменным
        grad_x1 = np.sin(2 * cur_x1)
        grad_x2 = np.sin(2 * cur_x2)
                 
        # после того, как посчитаны производные, можно обновить веса
        # не забудьте про lr
        cur_x1 -= lr * grad_x1
        cur_x2 -= lr * grad_x2

    return np.array(steps)

Запустим градиентный спуск:

In [ ]:
steps = grad_descent(lr=0.5, num_iter=10)

Визуализируем точки градиентного спуска на 3D-графике нашей функции. Кружочками будут обозначены точки (тройки $x_1, x_2, f(x_1, x_2)$), по которым ваш алгоритм градиентного спуска двигался к минимуму.

Для того, чтобы нарисовать этот график, мы и сохраняли значения $cur\_x_1, cur\_x_2, f(cur\_x_1, cur\_x_2)$ в `steps` в процессе спуска.

Если у вас правильно написана функция `grad_descent_2d`, то кружочки на картинке должны сходиться к одной из точек минимума функции. Вы можете менять начальные приближения алгоритма, значения `lr` и `num_iter` и получать разные результаты.

In [ ]:
# %matplotlib osx

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

path = []

X, Y = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))

fig = plt.figure(figsize=(16, 10))
ax = fig.gca(projection='3d')

# редактируем настройки отображения траектории, сделанной градиентым спуском
ax.plot(xs=steps[:, 0], ys=steps[:, 1], zs=steps[:, 2],
        marker='o', markersize=8, zorder=3, 
        markerfacecolor='red', lw=3, c='green')

ax.plot_surface(X, Y, f(X, Y), cmap=cm.coolwarm)
ax.set_zlim(0, 5)
ax.view_init(elev=60)
plt.show()

## Линейные модели

Возьмём код для линейной регрессии с семинара. Напомним, что найти веса $W$ и $b$ для линейной регрессии можно двумя способами: обращением матриц (функция `solve_weights`) и градиентным спуском (функция `grad_descent`). Мы здесь будем рассматривать градиентный спуск.

In [ ]:
W = None
b = None


def mse(y_pred, y):
    return np.mean((y_pred - y) ** 2)


def grad_descent(X, y, lr, num_iter=100):
    global W, b
    
    np.random.seed(40)
    
    W = np.random.rand(X.shape[1])
    b = np.array(np.random.rand(1))
    
    losses = []
    
    N = X.shape[0]
    
    for iter_num in range(num_iter):
        y_pred = predict(X)
        losses.append(mse(y_pred, y))
        
        W_grad = np.zeros_like(W)
        b_grad = 0
        
        for sample, prediction, label in zip(X, y_pred, y):
            W_grad += 2 * (prediction - label) * sample
            b_grad += 2 * (prediction - label)
        
        W -= lr * W_grad
        b -= lr * b_grad
    
    return losses


def predict(X):
    global W, b
    return np.squeeze(X @ W + b.reshape(-1, 1))

Рассмотрим функцию:  

$$
f(x, y) = 0.43x+0.5y + 0.67
$$  

**Напоминание.** Что мы хотим? Мы хотим уметь "восстанавливать функцию" - то есть предсказывать значения функции в точках с координатами $(x, y)$ (именно так и получается 3D-график - $(x, y, f(x,y))$ в пространстве). В чём сложность? Нам дан только конечный небольшой набор точек ($30$ в данном случае), по которому мы хотим восстановить зависимость, по сути, непрерывную. Линейная регрессия как раз подходит для восстановления линейной зависимости (а у нас функция сейчас как раз линейная - только сложение аргументов и умножение на число).

Cгерерируем шумные данные из этой функции (как на семинаре):

In [ ]:
np.random.seed(40)
func = lambda x, y: (0.43 * x + 0.5 * y + 0.67 + np.random.normal(0, 7, size=x.shape))

X = np.random.sample(30) * 10
Y = np.random.sample(30) * 150

result_train = [func(x, y) for x, y in zip(X, Y)]
data_train = np.concatenate([X.reshape(-1, 1), Y.reshape(-1, 1)], axis=1)

pd.DataFrame({'x': X, 'y': Y, 'res': result_train}).head()

Посмотрим, что же мы сгенерировали:

In [ ]:
# %matplotlib osx

fig = plt.figure(figsize=(16, 10))
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 150, 100))
ax.scatter(xs=data_train[:, 0], ys=data_train[:, 1], zs=result_train, c='red')
ax.plot_surface(X,Y, 0.43*X + 0.5*Y + 0.67, color='green', alpha=0.3)

ax.view_init(elev=60)
plt.ion()

Теперь давайте попробуем применить к этим данным нашу линейную регрессию и с помощью неё предсказать истинный график функции:

In [ ]:
losses = grad_descent(data_train, result_train, 1e-2, 5)

In [ ]:
W, b

Посмотрим на график лосса:

In [ ]:
plt.plot(losses)
plt.show()

И на полученную разделяющую плоскость:

In [ ]:
# %matplotlib osx

fig = plt.figure(figsize=(16, 10))
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 150, 100))
ax.scatter(xs=data_train[:, 0], ys=data_train[:, 1], zs=result_train, c='red')
ax.plot_surface(X, Y, 0.43 * X + 0.5 * Y + 0.67, color='green', alpha=0.3)
ax.plot_surface(X,Y, W[0] * X + W[1] * Y + b, color='blue', alpha=0.3)

ax.view_init(elev=60)

Зелёная плоскость - истинная функция, синяя плоскость - предсказание.

О нет, лосс и коэффициенты ($W$ и $b$) нашей модели быстро уходит в небеса, и график предсказан неправильно. Почему такое происходит?

В данном случае дело в том, что признаки имеют разный **масштаб** (посмотрите на значения $X$ и $Y$ - они лежат в разных диапазонах). Многие модели машинного обучения, в том числе линейные, будут плохо работать в таком случае (на самом деле это зависит от метода оптимизации, сейчас это градиентный спуск).  

Есть несколько способов **масштабирования**:

1. **Нормализация** (она же **стандартизация**, `StandardScaling`):  

   $$
   x_{ij} = \frac{x_{ij} - \mu_j}{\sigma_j}
   $$
   
   где $j$ - номер признака, $i$ - номер объекта.
   
   То есть вычитаем среднее по столбцу и делим на корень из дисперсии.


2. **Приведение к отрезку $[0,1]$** (`MinMaxScaling`):  

   $$
   x_{ij} = \frac{x_{ij} - \min_j}{\max_j - \min_j}
   $$
   
   где $j$ - номер признака, $i$ - номер объекта.
   
   То есть вычитаем минимум по столбцу и делим на разницу между минимумом и максимумом.

### Нормализация

Посмотрим на среднее и разброс значений в признаках (координатах) до масштабирования:

In [ ]:
data_train.mean(axis=0)

In [ ]:
data_train.std(axis=0)

То есть в первом столбце у нас среднее $5.6$ и среднеквадратичное отклонение $2.7$, а во втором столбце среднее $70.8$ и среднеквадратичное отклонение $45.3$.  

Будьте внимательны: среднеквадратичное отклонение **НЕ** говорит о том, каков **максимальный** разброс, оно лишь указывает на числовой интервал, вероятность попадания в который у данного признака высока (то есть, например, в интервал $[2.9, 8.3]$ в первом случае). Для большей информации смотрите [среднеквадратичное отклонение](https://ru.wikipedia.org/wiki/Среднеквадратическое_отклонение) и [доверительные интервалы](https://ru.wikipedia.org/wiki/Доверительный_интервал).

Однако факт в том, что масштаб у этих признаков разный, давайте исправим это.

Нормализуйте признаки так, чтобы среднее значение в каждом столбце было $\sim 0$, а стандартное отклонение $\sim 1$:

In [ ]:
data_train_normalized = (data_train - data_train.mean(axis=0)) / data_train.std(axis=0)

Проверьте средние и диспресию:

In [ ]:
data_train_normalized.mean(axis=0)

In [ ]:
data_train_normalized.std(axis=0)

Попробуем теперь запустить регрессию с теми же параметрами `lr` и `num_iter`:

In [ ]:
losses = grad_descent(data_train_normalized, result_train, 1e-2, 100)

In [ ]:
W, b

In [ ]:
plt.plot(losses)
plt.show()

Мы видим, что теперь коэффициенты по модулю не огромны, градиентный спуск не взрывается и лосс стабилен!

Посмотрим на полученную плоскость:

In [ ]:
# %matplotlib osx

fig = plt.figure(figsize=(16, 10))
ax = fig.gca(projection='3d')

ax.scatter(xs=data_train[:, 0], ys=data_train[:, 1], zs=result_train, c='red')

X, Y = np.meshgrid(np.linspace(-1, 10, 100), np.linspace(-1, 150, 100))
ax.plot_surface(X, Y, 0.43 * X + 0.5 * Y + 0.67, color='green', alpha=0.3)

X, Y = np.meshgrid(np.linspace(-1, 10, 100), np.linspace(-1, 150, 100))
# не забудьте нормализовать X и Y тоже
ax.plot_surface(X, Y, W[0] * (X - X.mean()) / X.std() + W[1] * (Y - Y.mean()) / Y.std() + b, color='blue', alpha=0.3)

ax.view_init(elev=60)
plt.ion()

### Регуляризация

Помимо "сырой" линейной регрессии часто используют линейную регрессию с регуляризацией - **Lasso** или **Ridge** регрессию. Они отличаются только типом "штрафа" за большие веса: учитывать модули (**Lasso**) или квадраты весов (**Ridge**).

Учитывая, что наш лосс в этой задаче - Mean Squared Error (MSE), в случае Ridge-регрессии будет:  

$$
Loss = MSE = \sum (pred_i-y_i)^2 + \alpha*\sum W_i^2
$$  

А в случае Lasso-регрессии будет:  

$$
Loss = MSE = \sum (pred_i-y_i)^2 + \alpha*\sum |W_i|
$$  

Здесь $\alpha$ - заранее задаваемый гиперпараметр. Это вес, с которым второе слагаемое будет влиять на лосс.

Добавление регуляризации, как правило, помогает бороться с **переобучением**.

Подробнее об этом можно почитать [тут](http://www.machinelearning.ru/wiki/index.php?title=Лассо) и [тут](http://www.machinelearning.ru/wiki/index.php?title=Ридж-регрессия).

## Полезные ссылки

1. Открытый курс по машинному обучению - https://habr.com/company/ods/blog/323890/

2. Если вам интересно математическое обоснование всех методов, рекомендуем ознакомиться с этой книгой - https://web.stanford.edu/~hastie/ElemStatLearn/printings/ESLII_print12.pdf